# Habituation to a toy model of odor background
Look at new odor recognition on top of a two-odor, one fluctuating proportion background:

$$ \vec{x}(t) = \left(\frac12 + \nu \right) \vec{x}_a + \left(\frac12 - \nu \right) \vec{x}_b $$

The odor vectors are generated with i.i.d. exponential elements with scale (mean) mu=0.1, then normalized to have an L2 norm of 1. I take $\nu(t)$ following a Ornstein-Uhlenbeck process, typically with standard deviation $\sigma = 0.3$ and average $\langle \nu \rangle = 0$. 

Here, I optionally clip elements of $\vec{x}(t)$ to be non-negative, although this does not make a significant difference since samples where $|\nu| > 0.5$ are rare. 

## Imports

In [ ]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from modelfcts.ibcm import (
    integrate_inhib_ibcm_network_options,
    ibcm_respond_new_odors,
    compute_mbars_cgammas_cbargammas,
    ibcm_respond_new_odors
)
from modelfcts.ibcm_analytics import (
    fixedpoints_m_2vectors, 
    fixedpoints_barm_2vectors, 
    fixedpoints_w_2vectors, 
    fixedpoint_s_2vectors_instant, 
    fixedpoint_s_2vectors_mean, 
    fixedpoint_s_2vectors_norm2
)
from modelfcts.biopca import (
    integrate_inhib_ifpsp_network_skip,
    build_lambda_matrix,
    biopca_respond_new_odors
)
from modelfcts.average_sub import (
    integrate_inhib_average_sub_skip, 
    average_sub_respond_new_odors
)
from modelfcts.ideal import(
    find_projector, 
    find_parallel_component, 
    ideal_linear_inhibitor
)
from modelfcts.checktools import (
    compute_pca_meankept, 
    compute_projector_series, 
    analyze_pca_learning
)
from modelfcts.backgrounds import (
    update_ou_2inputs_clip, 
    update_ou_2inputs,
    logof10, 
    decompose_nonorthogonal_basis, 
    generate_odorant
)
from modelfcts.tagging import (
    project_neural_tag, 
    create_sparse_proj_mat, 
    SparseNDArray, 
    tags_list_to_csr_matrix
)
from utils.statistics import seed_from_gen
from utils.smoothing_function import (
    moving_average, 
    moving_var
)
from simulfcts.plotting import (
    plot_cbars_gammas_sums, 
    plot_cbars_gamma_series, 
    plot_3d_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats
from utils.metrics import jaccard

In [ ]:
from utils.metrics import l2_norm, l1_norm, linf_norm, cosine_dist

def distance_panel_target(mixes, target):
    """ Compute a panel of distances between the pure (target) new odor and mixtures 
    (which can be without inhibition, with average inhibition, IBCM inhibition, etc.). 
    
    Four distances included, in order: l2, l1, linf, cosine_dist
    
    Args:
        mixes (np.ndarray): mixtures of odors to compute distance from target, 
            the last axis should have the size of target, 
            while other axes are arbitrary.  
        target (np.1darray): target odor vector, same length as
            last axis of mixes. 
    Returns:
        dist_panel (np.ndarray): shape of pure, except the last axis, 
            which has length 4 (for the number of distances computed). 
    """
    # Make axis 0 the axis indexing distance metrics, to begin with
    # And move it to the last axis before returning
    dist_array = np.zeros([4] + list(mixes.shape[:-1]))
    # No need to add axes to target vector; if it is 1d, it is broadcasted
    # along the last axis of mixes, which indexes elements of each vector. 
    dist_array[0] = l2_norm(target - mixes)
    dist_array[1] = l1_norm(target - mixes)
    dist_array[2] = linf_norm(target - mixes)
    dist_array[3] = cosine_dist(target, mixes)
    
    return np.moveaxis(dist_array, 0, -1)

### Aesthetic parameters

In [ ]:
#plt.style.use(['dark_background'])
plt.rcParams["figure.figsize"] = (4.5, 3.0)

In [ ]:
models = ["ibcm", "biopca", "avgsub", "ideal", "orthogonal", "none"]
model_nice_names = {
    "ibcm": "IBCM",
    "biopca": "BioPCA",
    "avgsub": "Average",
    "ideal": "Ideal",
    "orthogonal": "Orthogonal",
    "none": "None"
}
model_colors = {
    "ibcm": "xkcd:turquoise",
    "biopca": "xkcd:orangey brown",
    "avgsub": "xkcd:navy blue",
    "ideal": "xkcd:powder blue",
    "orthogonal": "xkcd:pale rose",
    "none": "grey"
}

### Initialization

In [ ]:
# Initialize common simulation parameters
n_dimensions = 25
n_components = 2  # Two odors but only one fluctuating proportion

inhib_rates = [0.00025, 0.00005]  # alpha, beta

# Simulation duration
duration = 80000.0
deltat = 1.0
skp = 1  # We can save all time steps, small enough network

# Common model options
activ_function = "identity"  #"ReLU"

# Background process
update_fct = update_ou_2inputs

# Choose randomly generated background vectors
rgen_meta = np.random.default_rng(seed=0x4146e7d68791560a406669280c37e5bb)

back_components = np.zeros([n_components, n_dimensions])
for i in range(n_components):
    back_components[i] = generate_odorant(n_dimensions, rgen_meta, lambda_in=0.1)
back_components = back_components / l2_norm(back_components).reshape(-1, 1)

# Seed for background simulation, to make sure all models are the same
simul_seed = seed_from_gen(rgen_meta)

# Initial background vector and initial nu values
average_nu = np.zeros(1)
init_nu = np.zeros(1)
init_bkvec = 0.5*back_components[0] + 0.5*back_components[1]
# nus are first in the list of initial background params
init_back_list = [init_nu, init_bkvec]

## Compute the coefficients in the Ornstein-Uhlenbeck update equation
sigma2_nu = 0.09
tau_nu = 2.0  # Fluctuation time scale of the background nu_alphas (same for all)
update_coefs_mean = np.exp(-deltat/tau_nu)
update_coefs_noise = np.sqrt(sigma2_nu*(1 - np.exp(-2*deltat/tau_nu)))

back_params = [average_nu, update_coefs_mean, update_coefs_noise, back_components]

## IBCM habituation
### IBCM simulation

In [ ]:
# IBCM model parameters
n_i_ibcm = 2  # Two neurons is enough, one per fixed point x_{\pm}

learnrate_ibcm = 0.0025
tau_avg_ibcm = 300
coupling_eta_ibcm = 0.2
decay_relative_ibcm = 0.005
ibcm_rates = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    None,   # Saturation, none
    None,   # For the Law and Cooper variant
    decay_relative_ibcm
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "linear", 
    "variant": "intrator", 
    "decay": True
}

# Initial synaptic weights: small positive noise
init_synapses_ibcm = 0.1*rgen_meta.random(size=[n_i_ibcm, n_dimensions])

In [ ]:
# Run the IBCM simulations
simul_seed = seed_from_gen(rgen_meta)
sim_results = integrate_inhib_ibcm_network_options(
                init_synapses_ibcm, update_fct, init_back_list, 
                ibcm_rates, inhib_rates, back_params, duration, 
                deltat, seed=simul_seed, noisetype="normal",  
                skp=skp, **ibcm_options
)

(tser_ibcm, 
 nuser_ibcm, 
 bkvecser_ibcm, 
 mser_ibcm, 
 cbarser_ibcm, 
 wser_ibcm, 
 sser_ibcm) = sim_results

### IBCM habituation analysis

In [ ]:
init_synapses_ibcm

In [ ]:
# Calculate cgammas_bar and mbars
transient = 50000 // skp
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser, c_gammas, cbars_gamma = compute_mbars_cgammas_cbargammas(
                                    mser_ibcm, coupling_eta_ibcm, back_components)
sums_cbars_gamma = np.sum(cbars_gamma, axis=2)
sums_cbars_gamma2 = np.sum(cbars_gamma*cbars_gamma, axis=2)

# Analytical prediction of fixed points
analytical_barm, _ = fixedpoints_barm_2vectors(back_components, np.sqrt(sigma2_nu), 
                                               coupling_eta_ibcm, n_r=n_dimensions)
# Plot dot products with x_a and x_b, in terms of reduced m vectors. 
# Take any of the fixed points for one neuron, its two dot products are the only two possible dot product values. 
c_specif = np.dot(analytical_barm[2, 0], back_components[0])
c_nonspecif = np.dot(analytical_barm[2, 0], back_components[1])

In [ ]:
# Plot the cbar2_avg term throughout
cbar2_avg_ser = moving_average(cbarser_ibcm*cbarser_ibcm, kernelsize=tau_avg_ibcm)
neurons_cmap = sns.color_palette("Greys", n_colors=n_i_ibcm)
fig, ax = plt.subplots()
for i in range(n_i_ibcm):
    ax.plot(tser_ibcm[:-tau_avg_ibcm], cbar2_avg_ser[:-tau_avg_ibcm, i], 
            color=neurons_cmap[i])
ax.set(xlabel="Time (x1000)", ylabel=r"$\bar{c}^2$ moving average")
plt.show()
plt.close()

In [ ]:
fig , ax, _ = plot_cbars_gamma_series(tser_ibcm, cbars_gamma, 
                        skp=10, transient=50000 // skp)
# Compare to exact analytical fixed point solution
ax.axhline(c_specif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{specific}}$")
ax.axhline(c_nonspecif, ls="-.", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{non}}$")
ax.legend()
plt.show()
plt.close()

In [ ]:
# Correlation between nu's and c's, see if some neurons are specific to odors
# Each neuron turns out to correlate its response to  one concentration
# that means it is specific to that odor. 
cbarser_norm_centered = cbarser_ibcm - np.mean(cbarser_ibcm[transient:], axis=0)
conc_ser_centered = np.stack([nuser_ibcm[:, 0], -nuser_ibcm[:, 0]], axis=1)
correl_c_nu = np.mean(cbarser_norm_centered[transient:, :, None] 
                      * conc_ser_centered[transient:, None, :], axis=0)

fig, ax = plt.subplots()
img = ax.imshow(correl_c_nu.T)
ax.set(ylabel=r"Component $\gamma$", xlabel=r"Neuron $i$")
fig.colorbar(img, label=r"$\langle (\bar{c}^i - \langle \bar{c}^i \rangle)"
             r"(\nu_{\gamma} - \langle \nu_{\gamma} \rangle) \rangle$", 
            location="top")
plt.show()
plt.close()

# Check if each component has at least one neuron
split_val = 0.0
for comp in range(n_components):
    print("Number of neurons specific to component {}: {}".format(
            comp, np.sum(np.mean(cbars_gamma[-2000:, :, comp], axis=0) > split_val)))

In [ ]:
fig, axes, _ = plot_background_neurons_inhibition(tser_ibcm, bkvecser_ibcm, sser_ibcm, skp=10)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, snorm_ser = plot_background_norm_inhibition(
                                tser_ibcm, bkvecser_ibcm, sser_ibcm, skp=10)

# Compute noise reduction factor, annotate
transient = 50000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, snorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_ibcm, wser_ibcm, skp=100)
plt.show()
plt.close()

## BioPCA simulation
### BioPCA habituation simulation

In [ ]:
# BioPCA model parameters
n_i_pca = 1  # Number of inhibitory neurons for BioPCA case

# Model rates
learnrate_pca = 0.0005  # Learning rate of M
rel_lrate_pca = 2.0  # Learning rate of L, relative to learnrate
# Choose Lambda diagonal matrix as advised in Minden et al., 2018
lambda_range_pca = 0.5
lambda_mat_diag = build_lambda_matrix(lambda_range_pca, n_i_pca)
xavg_rate_pca = learnrate_pca
pca_options = {
    "activ_fct": activ_function, 
    "remove_lambda": False, 
    "remove_mean": True
}
biopca_rates = [learnrate_pca, rel_lrate_pca, lambda_range_pca]
if pca_options["remove_mean"]:
    biopca_rates.append(xavg_rate_pca)


# Initial synaptic weights: small positive noise
init_synapses_pca = rgen_meta.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_i_pca)
init_mmat_pca = rgen_meta.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_dimensions)
init_lmat_pca = np.eye(n_i_pca, n_i_pca)  # Supposed to be near-identity, start as identity
ml_inits_pca = [init_mmat_pca, init_lmat_pca]

In [ ]:
# Run simulation
sim_results = integrate_inhib_ifpsp_network_skip(
                ml_inits_pca, update_fct, init_back_list, biopca_rates, 
                inhib_rates, back_params, duration, deltat, 
                seed=simul_seed, noisetype="normal", skp=skp, **pca_options)
(tser_pca, 
 nuser_pca, 
 bkvecser_pca, 
 mser_pca, 
 lser_pca, 
 xser_pca, 
 cbarser_pca, 
 wser_pca, 
 sser_pca) = sim_results

### BioPCA simulation analysis

#### Analytical predictions for one PCA neuron
$L$ matrix: 1x1, a scalar, equal to the inverse of the eigenvalue: $L = 1/L' = \frac{1}{\sigma^2 \| \vec{x}_s \|^2}$

$M$ matrix: 1x$n_R$, parallel to the fluctuating part of the background, $\vec{m} = \sigma^2 \| \vec{x}_s \| \vec{x}_s$.

$W$ matrix: $n_Rx1$, a vector also parallel to $\vec{x}_s$, we find $\vec{w} = \frac{\sigma^2 \| \vec{x}_s \|}{\sigma^2 \| \vec{x}_s \|^2 + \beta/\alpha} \vec{x}_s$. 

Then the instantaneous PN activity should be $\vec{s}(t) = \vec{x}(t) - \langle \vec{x} \rangle - WLM(\vec{x}(t) - \langle \vec{x} \rangle) = \frac{\beta/\alpha}{\beta/\alpha + \sigma^2 \| \vec{x}_s \|^2} \nu(t) \vec{x}_s$

In [ ]:
from modelfcts.pca_analytics import (
    fixedpoints_pca_2vectors, 
    pca_fixedpoint_s_2vectors_instant, 
    pca_fixedpoint_s_2vectors_variance
)

In [ ]:
x_s = (back_components[0] - back_components[1])
x_d = (back_components[0] + back_components[1])/2
m_pca_pred, l_pca_pred, w_pca_pred = fixedpoints_pca_2vectors(back_components, sigma2_nu, inhib_rates)
sser_pca_pred = pca_fixedpoint_s_2vectors_instant(back_components, sigma2_nu, inhib_rates, bkvecser_pca)
svari_pca_pred = pca_fixedpoint_s_2vectors_variance(back_components, sigma2_nu, inhib_rates)
res = analyze_pca_learning(bkvecser_pca, mser_pca, lser_pca, 
                           lambda_mat_diag, demean=pca_options["remove_mean"])
true_pca, learnt_pca, fser, off_diag_l_avg_abs, align_error_ser = res

In [ ]:
print(m_pca_pred)
print(l_pca_pred)
print(w_pca_pred)
print(sser_pca_pred[-1])
xvari_pca = np.var(l2_norm(bkvecser_pca, axis=1))
xvari_pca = np.mean(np.sum((bkvecser_pca - x_d)**2, axis=1), axis=0)
print(np.sqrt(svari_pca_pred / xvari_pca))

In [ ]:
from utils.statistics import principal_component_analysis
from modelfcts.checktools import compute_pca_meankept

In [ ]:
fig, axes = plot_pca_results(tser_pca/1000, true_pca, learnt_pca, align_error_ser, off_diag_l_avg_abs)
axes[-1].set_xlabel("Time (x1000 steps)")
fig.set_size_inches(fig.get_size_inches()[0], 3*2)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, snorm_ser = plot_background_norm_inhibition(
                                tser_pca, bkvecser_pca, sser_pca, skp=10)

# Compute noise reduction factor, annotate
transient = 50000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, snorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
fig, axes, _ = plot_background_neurons_inhibition(tser_pca, bkvecser_pca, sser_pca, skp=10)
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_pca, wser_pca, skp=10)
plt.show()
plt.close()

### Average background subtraction simulation

In [ ]:
# Average subtraction model parameters
avg_options = {"activ_fct": activ_function}

# Initial synaptic weights: dummy
init_synapses_avg = np.zeros([1, n_dimensions])

In [ ]:
sim_results = integrate_inhib_average_sub_skip(
                init_synapses_avg, update_fct, init_back_list, 
                [], inhib_rates, back_params, duration, deltat,
                seed=simul_seed, noisetype="normal", skp=skp, **avg_options
)
tser_avg, bkser_avg, bkvecser_avg, wser_avg, sser_avg = sim_results

### Ideal inhibition
The component parallel to the background is reduced to beta / (2*alpha + beta). 

In [ ]:
back_projector = find_projector(back_components.T)
ideal_factor = inhib_rates[1] / (2*inhib_rates[0] + inhib_rates[1])
sser_ideal = bkvecser_ibcm * ideal_factor

## Model comparison for background inhibition

In [ ]:
snorm_series = {
    "ibcm": l2_norm(sser_ibcm), 
    "biopca": l2_norm(sser_pca), 
    "avgsub": l2_norm(sser_avg), 
    "none": l2_norm(bkvecser_ibcm), 
    "ideal": l2_norm(sser_ideal)
}
std_options = dict(kernelsize=2001, boundary="free")
mean_options = dict(kernelsize=2001, boundary="free")
std_series = {
    a: np.sqrt(moving_var(snorm_series[a], **std_options)) for a in snorm_series
} 
mean_series = {
    a: moving_average(snorm_series[a], **mean_options) for a in snorm_series
}

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)
axes = axes.flatten()
for model in std_series.keys():
    props = dict(label=model_nice_names[model], color=model_colors[model])
    axes[0].plot(tser_ibcm / 1000, mean_series[model], **props)
    axes[1].plot(tser_ibcm / 1000, std_series[model], **props)
snorm_string = r"$\|\vec{s}\|$"
axes[0].set_ylabel(r"PN activity norm, " + snorm_string)
axes[1].set(xlabel="Time (x1000 steps)", ylabel=r"Standard deviation " + snorm_string)
axes[0].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
fig.set_size_inches(4.5, 2.5*2)
plt.show()
plt.close()

## Model comparison for new odor recognition

In [ ]:
def find_snap_index(dt, skip, times):
    """ Find nearest multiple of dt*skip to each time in times """
    return np.around(times / (dt*skip)).astype(int)

In [ ]:
# Common parameters
n_kc = 1000
projection_arguments = {
    "kc_sparsity": 0.05,
    "adapt_kc": True,
    "n_pn_per_kc": 3,
    "project_thresh_fact": 0.1
}
proj_mat = create_sparse_proj_mat(n_kc, n_dimensions, rgen_meta)

sser_dict = {
    "ibcm": sser_ibcm, 
    "biopca": sser_pca, 
    "avgsub": sser_avg, 
    "none": bkvecser_ibcm, 
    "ideal": sser_ideal
}

In [ ]:
# Generate new odors, select test times, etc.
# New odors tested
n_new = 100
new_odors = generate_odorant([n_new, n_dimensions], rgen_meta, lambda_in=0.1)
new_odors /= l2_norm(new_odors)[:, None]

# Test times
n_test_times = 10
start_test_t = duration - n_test_times * 2000.0
test_times = np.linspace(start_test_t, duration, n_test_times)
test_times -= deltat*skp
test_idx = find_snap_index(deltat, skp, test_times)

# New odor concentrations
new_test_concs = np.asarray([0.5, 1.0])
avg_whiff_conc = 0.5
print("Average whiff concentration: {:.4f}".format(avg_whiff_conc))
new_test_concs *= avg_whiff_conc
n_new_concs = len(new_test_concs)

# Background samples, indexed [time, sample, n_orn]
n_back_samples = 10
# Steady-state distribution of nu: normal distribution, mean 0, variance sigma2
conc_samples = rgen_meta.normal(loc=0.0, scale=np.sqrt(sigma2_nu), size=n_test_times*(n_back_samples-1))
# Clip between -0.5 and 0.5
conc_samples = np.clip(conc_samples, a_min=-0.5, a_max=0.5).reshape(-1, 1)
back_samples = (0.5+conc_samples)*back_components[0:1] + (0.5-conc_samples)*back_components[1:2]
back_samples = back_samples.reshape([n_test_times, n_back_samples-1, -1])
back_samples = np.concatenate([bkvecser_ibcm[test_idx, None, :], back_samples], axis=1)

# Containers for s vectors of each model
mixture_svecs = {a: np.zeros([n_new, n_test_times,  n_new_concs,  
                    n_back_samples, n_dimensions]) for a in sser_dict.keys()}
mixture_tags = {a: SparseNDArray((n_new, n_test_times, n_new_concs,
                    n_back_samples, n_kc), dtype=bool) for a in sser_dict.keys()}
new_odor_tags = sparse.lil_array((n_new, n_kc), dtype=bool)
jaccard_scores = {a: np.zeros([n_new, n_test_times, n_new_concs,  n_back_samples]) 
                  for a in sser_dict.keys()}

In [ ]:
for i in range(n_new):
    # Compute neural tag of the new odor alone, without inhibition
    new_tag = project_neural_tag(
                    new_odors[i], new_odors[i],
                    proj_mat, **projection_arguments
                )
    new_odor_tags[i, list(new_tag)] = True
    # Parallel and orthogonal components
    x_new_par = find_parallel_component(new_odors[i], 
                        back_components, back_projector)
    x_new_ort = new_odors[i] - x_new_par
    # Now, loop over snapshots, mix the new odor with the back samples,
    # compute the PN response at each test concentration,
    # compute tags too, and save results
    for j in range(n_test_times):
        jj = test_idx[j]
        for k in range(n_new_concs):
            mixtures = (back_samples[j]
                + new_test_concs[k] * new_odors[i])
            # odors, mlx, wmat, 
            # Compute for each model
            mixture_svecs["ibcm"][i, j, k] = ibcm_respond_new_odors(
                mixtures, mser_ibcm[jj], wser_ibcm[jj], 
                ibcm_rates, options=ibcm_options
            )
            mixture_svecs["biopca"][i, j, k] = biopca_respond_new_odors(
                mixtures, [mser_pca[jj], lser_pca[jj], xser_pca[jj]], 
                wser_pca[jj], biopca_rates, options=pca_options
            )
            mixture_svecs["avgsub"][i, j, k] = average_sub_respond_new_odors(
                mixtures, wser_avg[jj], options=avg_options
            )
            mixture_svecs["none"][i, j, k] = mixtures
            mixture_svecs["ideal"][i, j, k] = ideal_linear_inhibitor(
                x_new_par, x_new_ort, mixtures, new_test_concs[k], 
                *inhib_rates, **avg_options
            )
            for l in range(n_back_samples):
                for mod in mixture_svecs.keys():
                    mix_tag = project_neural_tag(
                        mixture_svecs[mod][i, j, k, l], mixtures[l],
                        proj_mat, **projection_arguments
                    )
                    try:
                        mixture_tags[mod][i, j, k, l, list(mix_tag)] = True
                    except ValueError as e:
                        print(mix_tag)
                        print(mixture_svecs[mod][i, j, k, l])
                        print(proj_mat.dot(mixture_svecs[mod][i, j, k, l]))
                        raise e
                    jaccard_scores[mod][i, j, k, l] = jaccard(mix_tag, new_tag)

In [ ]:
# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "ideal", "avgsub", "biopca", "ibcm"]
for m in models:  # Plot IBCM last
    all_jacs = jaccard_scores[m]
    for i in range(n_new_concs):
        hist_outline(
            axes[i], all_jacs[:, :, i, :].flatten(),
            bins="doane", density=True, label=model_nice_names.get(m, m),
            color=model_colors.get(m), alpha=1.0
        )
        axes[i].axvline(
            np.median(all_jacs[:, :, i, :]), ls="--",
            color=model_colors.get(m)
        )
# Labeling the graphs, etc.
for i in range(n_new_concs):
    ax = axes[i]
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel("Jaccard similarity (higher is better)")
    axes[i].set_ylabel("Probability density")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
#fig.savefig("figures/detection/compare_models_one_odor_habituation_{}.pdf".format(activ_function),
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()

In [ ]:
# Distance to new odor
# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "ideal", "avgsub", "biopca", "ibcm"]
all_medians = []
for m in models:  # Plot IBCM last
    all_distances = (mixture_svecs[m] 
         - new_test_concs[None, None, :, None, None]*new_odors[:, None, None, None, :])
    all_norms = l2_norm(all_distances.reshape(-1, n_dimensions))
    all_medians.append(np.median(all_norms))
    for i in range(n_new_concs):
        hist_outline(
            axes[i], all_norms,
            bins="doane", density=True, label=model_nice_names.get(m, m),
            color=model_colors.get(m), alpha=1.0
        )
        axes[i].axvline(
            all_medians[-1], ls="--",
            color=model_colors.get(m)
        )
# Labeling the graphs, etc.
for i in range(n_new_concs):
    axes[i].set_xlim([0.0, 2.0*max(all_medians)])
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel(r"Distance to new odor, $\|\vec{s} - \vec{x}_{\mathrm{new}}\|$")
    axes[i].set_ylabel("Probability density")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
#fig.savefig("figures/detection/compare_models_onerun_snorm_{}.pdf".format(activ_fct),
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()